In [2]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\ANACONDA\python.exe -m pip install --upgrade pip' command.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import cv2
import time
import imutils

In [4]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
imagePaths = list(paths.list_images('./input/face-mask-detection-data'))
data = []
labels = []

for imagePath in imagePaths:

    label = imagePath.split(os.path.sep)[-2]
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(label)
print("No. of images loaded: {}".format(len(data)))


data = np.array(data, dtype="float32")
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

D:\ANACONDA\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


No. of images loaded: 3833


In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = False

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)
    

D:\ANACONDA\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/20
95/95 [==============================] - 108s 1s/step - loss: 0.3855 - accuracy: 0.8724 - val_loss: 0.1720 - val_accuracy: 0.9752
Epoch 2/20
95/95 [==============================] - 101s 1s/step - loss: 0.1487 - accuracy: 0.9661 - val_loss: 0.0930 - val_accuracy: 0.9804
Epoch 3/20
95/95 [==============================] - 101s 1s/step - loss: 0.1008 - accuracy: 0.9763 - val_loss: 0.0667 - val_accuracy: 0.9817
Epoch 4/20
29/95 [========>.....................] - ETA: 59s - loss: 0.0837 - accuracy: 0.9741 

In [ ]:
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

In [ ]:
newpath = list(paths.list_images('./my_input/withmask'))

newpath = newpath[0]
img = load_img(newpath)
img = img_to_array(img)
img = cv2.resize(img,(224,224))
img = img.reshape(1,224,224,3)

img = preprocess_input(img)
#print(img.shape)
img = np.array(img, dtype="float32")
pred = model.predict(img)
#print(pred)
if(pred[0][0] > pred[0][1]):
    print("mask detected")
else:
    print("mask not detected")